In [ ]:
from google.colab import drive

drive.mount('/content/drive')

ValueError: Mountpoint must not already contain files

In [ ]:
!pip install -q flask pyngrok huggingface-hub
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124 --force-reinstall --no-deps
!pip install typing-extensions numpy diskcache jinja2

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 3.3 MB/s eta 0:00:00


In [ ]:
!nvidia-smi


Thu Nov 20 10:32:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from pyngrok import ngrok
import re
import time
import llama_cpp
from flask import Flask, request, jsonify
import subprocess
print(llama_cpp.llama_supports_gpu_offload())


# Load model
llm = llama_cpp.Llama.from_pretrained(
    repo_id="VibeStudio/Nidum-Llama-3.2-3B-Uncensored-GGUF",
    filename="model-Q6_K.gguf",
    cache_dir="/content/drive/MyDrive/models",
    n_ctx=2048,
    n_threads=2,
    n_gpu_layers=-1,
    verbose=False
)
print(f"Model metadata: {llm.metadata}")

print("Model loaded successfully")

# Set up Flask API
ngrok.set_auth_token("")

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    print("Got the request now.")
    start_time = time.time()
    data = request.json
    prompt = data.get('prompt', '')
    craziness_meter = data.get('craziness', 0)

    if not prompt:
        return jsonify({'error': 'No prompt provided'}), 400

    response = llm(
        prompt,
        max_tokens=40,
        temperature=0.3 + (float(craziness_meter) * 0.7),  # Fixed: added comma
        top_p=0.95,
        repeat_penalty=1.1,
        seed=None,
        stop=["\n", "Patient:", "<>", "#", "Type something"]  # Stop at these tokens
    )

    result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used,utilization.gpu', '--format=csv,noheader,nounits'],
                          capture_output=True, text=True)
    print(f"GPU Usage: {result.stdout.strip()}")
    print(f"Took {time.time() - start_time} to get a response")

    generated_text = response['choices'][0]['text'].strip()

    print(f"Took {time.time() - start_time} to send")
    return jsonify({
        'generated_text': generated_text
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'ready'})

public_url = ngrok.connect(5000)
print(f"\nAPI is running at: {public_url}")
print("Copy this URL to your local Python file\n")

app.run(port=5000)

True


llama_context: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


Model metadata: {'general.quantization_version': '2', 'tokenizer.chat_template': '{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- if strftime_now is defined %}\n        {%- set date_string = strftime_now("%d %b %Y") %}\n    {%- else %}\n        {%- set date_string = "26 Jul 2024" %}\n    {%- endif %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if tools

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Got the request now.


...

GPU Usage: 3126, 83
Took 0.709632158279419 to get a response
Took 0.7096555233001709 to send
Got the request now.


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 12:25:46] "POST /generate HTTP/1.1" 200 -


GPU Usage: 3126, 83
Took 0.6518418788909912 to get a response
Took 0.651860237121582 to send
Got the request now.


INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 12:26:08] "POST /generate HTTP/1.1" 200 -


GPU Usage: 3126, 84
Took 0.5438077449798584 to get a response
Took 0.5438261032104492 to send
